In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np
import pandas as pd

# Loading FinBERT
tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-pretrain")
model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-pretrain", num_labels=1)

# Sliding window function with attention mask and batch processing
def predict_long_text(model, tokenizer, text, max_len=512, stride=256, aggregation='mean'):
    inputs = tokenizer(text, return_tensors="pt", truncation=False)
    input_ids = inputs["input_ids"][0]

    # Handle short inputs
    if len(input_ids) <= max_len:
        outputs = model(input_ids=input_ids.unsqueeze(0), attention_mask=torch.ones_like(input_ids).unsqueeze(0))
        return outputs.logits.item()

    scores = []
    chunks = []
    attention_masks = []

    for i in range(0, len(input_ids), stride):
        chunk = input_ids[i:i + max_len]
        attention_mask = torch.ones_like(chunk)

        if len(chunk) < max_len:
            padding = torch.zeros(max_len - len(chunk), dtype=torch.long)
            chunk = torch.cat((chunk, padding))
            attention_mask = torch.cat((attention_mask, padding))

        chunks.append(chunk)
        attention_masks.append(attention_mask)

    # Stack chunks for batch processing
    chunks = torch.stack(chunks)
    attention_masks = torch.stack(attention_masks)

    with torch.no_grad():
        outputs = model(input_ids=chunks, attention_mask=attention_masks)
        scores = outputs.logits.squeeze(-1).tolist()

    # Custom aggregation on the outputs
    if aggregation == 'mean':
        return np.mean(scores)
    elif aggregation == 'median':
        return np.median(scores)
    elif aggregation == 'max':
        return np.max(scores)
    else:
        raise ValueError("Invalid aggregation method. Use 'mean', 'median', or 'max'.")

# Applying the sentiment analisys on each cell
def predict_on_dataframe(df, text_column, model, tokenizer, aggregation='mean'):
    df['prediction'] = df[text_column].apply(lambda text: predict_long_text(model, tokenizer, text, aggregation=aggregation))
    return df


config.json:   0%|          | 0.00/359 [00:00<?, ?B/s]

C:\Users\dittr\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\dittr\.cache\huggingface\hub\models--yiyanghkust--finbert-pretrain. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at yiyanghkust/finbert-pretrain and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Reading the data
df = pd.read_csv("articles_translated.csv")

# Sentiment analisys
result_df = predict_on_dataframe(df, 'text_eng', model, tokenizer)

     Unnamed: 0                                               link  \
0             1  https://hang.hu/gazdasag/maris-kineveztek-varg...   
1             2  https://hang.hu/gazdasag/tobb-mint-haromezer-m...   
2             3  https://hang.hu/gazdasag/fokozza-az-inflaciot-...   
3             4  https://hang.hu/gazdasag/a-kozgazdaszok-szerin...   
4             5  https://hang.hu/gazdasag/recesszio-ide-vagy-od...   
..          ...                                                ...   
268         269  https://hang.hu/gazdasag/uj-melypontra-kuldte-...   
269         270  https://hang.hu/gazdasag/uj-korszak-a-rezsicso...   
270         271  https://hang.hu/gazdasag/gulyas-gergely-kata-1...   
271         272  https://hang.hu/gazdasag/allami-penzhez-juthat...   
272         273  https://hang.hu/gazdasag/kata-a-penzugyminiszt...   

                                                 title  \
0    Máris kinevezték Varga Mihályt a jegybank elnö...   
1    Több mint háromezer milliárd a költség

In [9]:
result_df

,Unnamed: 0,link,title,text,date,name,text_eng,prediction
0,1,https://hang.hu/gazdasag/maris-kineveztek-varg...,Máris kinevezték Varga Mihályt a jegybank elnö...,Sulyok Tamás köztársasági elnök kinevezte 2025...,2024-12-24,varga,President Tamás Sulyok has appointed Mihály Va...,0.145503
1,2,https://hang.hu/gazdasag/tobb-mint-haromezer-m...,Több mint háromezer milliárd a költségvetési h...,November végéig az államháztartás központi alr...,2024-12-09,varga,"By the end of November, the central sub-system...",0.143812
2,3,https://hang.hu/gazdasag/fokozza-az-inflaciot-...,Fokozza az inflációt az inflációkövető adóemel...,"– Ahogy az mindenki számára nyilvánvaló lehet,...",2024-10-31,varga,"- As should be obvious to everyone, the tax hi...",0.224084
3,4,https://hang.hu/gazdasag/a-kozgazdaszok-szerin...,A közgazdászok szerint legfeljebb 2 százalékos...,Az elmúlt napokban a kormány több tagja is bes...,2024-10-21,varga,"In recent days, several members of the governm...",0.292197
4,5,https://hang.hu/gazdasag/recesszio-ide-vagy-od...,"Recesszió ide vagy oda, Varga Mihály optimista","Továbbra is bővül a magyar gazdaság, a növeked...",2024-07-30,varga,"The Hungarian economy continues to expand, and...",0.117006
...,...,...,...,...,...,...,...,...
268,269,https://hang.hu/gazdasag/uj-melypontra-kuldte-...,"Miközben ment a kormányinfó, új mélypontra ker...","Ötszáz forint fölé erősödött az angol font, po...",2022-10-13,kinfo,The British pound strengthened to over five hu...,-0.049319
269,270,https://hang.hu/gazdasag/uj-korszak-a-rezsicso...,Új korszak a rezsicsökkentésben: mekkora száml...,A rezsicsökkentés új korszakának pontos szabál...,2022-07-14,kinfo,The exact rules for the new era of the cuts ha...,0.316882
270,271,https://hang.hu/gazdasag/gulyas-gergely-kata-1...,Gulyás Gergely: Nem látjuk indokoltnak az elfo...,a katázók kétharmada munkaviszonyt álcáz – fől...,2022-07-13,kinfo,two thirds of the street workers are disguised...,0.109531
271,272,https://hang.hu/gazdasag/allami-penzhez-juthat...,Állami pénzhez juthat a Wizz Air,A fapados légitársaság vezérigazgatója rendre ...,2020-07-09,kinfo,The CEO of the low-cost airline has consistent...,0.262625


In [ ]:
result_df.to_csv( "articles_tranlated_w_sentiment.csv")